In [5]:
from keras.models import Sequential
from keras.layers import Conv1D, GlobalMaxPooling1D, Embedding, LSTM
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer
from keras import metrics, regularizers
from keras_preprocessing.sequence import pad_sequences
from keras.metrics import categorical_accuracy
from keras.layers import InputLayer, Conv1D, Dense, Flatten, MaxPooling1D
import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import os

ModuleNotFoundError: No module named 'path'

In [6]:
filename = 'IT_Job_Data_Clean.csv'
data = pd.read_csv(filename, encoding= 'unicode_escape')

In [7]:
#Split the dataset to Training and Test subsets
train, test = train_test_split(data, train_size=0.9, test_size=0.1, random_state=42) #random_state = None

train_descs = train['Require']
train_labels = train['JobTitle']

test_descs = test['Require']
test_labels = test['JobTitle']

In [8]:
# Model Parameters
vocab_size = 100

sequences_length = 120

embedding_dimensionality = 32 #possibly low??
max_features = 100 #equal to vocab_size

num_labels = len(train_labels.unique())
batch_size = 32
nb_epoch = 20

nof_filters = 200
kernel_size = 16

hidden_dims = 512

In [9]:
# Convert Texts to Numeric Vectors for Input
tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(train_descs)

x_train = tokenizer.texts_to_sequences(train_descs)
x_test = tokenizer.texts_to_sequences(test_descs)

x_train = pad_sequences(x_train, maxlen = sequences_length, padding = 'post')
x_test = pad_sequences(x_test, maxlen = sequences_length, padding = 'post')

encoder = LabelBinarizer()
encoder.fit(train_labels)
y_train = encoder.transform(train_labels)
y_test = encoder.transform(test_labels)

In [10]:
from keras.layers import InputLayer, Conv1D, Dense, Flatten, MaxPooling1D

model = Sequential()
model.add(Embedding(max_features, embedding_dimensionality, input_length = 120))
model.add(Conv1D(nof_filters, kernel_size, padding='valid', activation='relu', strides = 1))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_labels, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer='adam', #'sgd', 'adam', 'RMSprop', 'Adagrad'
              metrics = [categorical_accuracy, 'accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 32)           3200      
                                                                 
 conv1d (Conv1D)             (None, 105, 200)          102600    
                                                                 
 activation (Activation)     (None, 105, 200)          0         
                                                                 
 max_pooling1d (MaxPooling1  (None, 52, 200)           0         
 D)                                                              
                                                                 
 lstm (LSTM)                 (None, 52, 128)           168448    
                                                                 
 dropout (Dropout)           (None, 52, 128)           0         
                                                        

In [11]:
history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=100,
    validation_data=(x_test, y_test),
)

Epoch 1/100
68/68 [==============================] - 6s 70ms/step - loss: 3.4664 - categorical_accuracy: 0.0814 - accuracy: 0.0814 - val_loss: 3.2404 - val_categorical_accuracy: 0.1286 - val_accuracy: 0.1286
Epoch 2/100
68/68 [==============================] - 5s 67ms/step - loss: 3.2623 - categorical_accuracy: 0.0893 - accuracy: 0.0893 - val_loss: 2.9082 - val_categorical_accuracy: 0.1743 - val_accuracy: 0.1743
Epoch 3/100
68/68 [==============================] - 4s 62ms/step - loss: 2.9466 - categorical_accuracy: 0.1314 - accuracy: 0.1314 - val_loss: 2.7323 - val_categorical_accuracy: 0.1826 - val_accuracy: 0.1826
Epoch 4/100
68/68 [==============================] - 4s 61ms/step - loss: 2.7395 - categorical_accuracy: 0.1869 - accuracy: 0.1869 - val_loss: 2.5537 - val_categorical_accuracy: 0.2407 - val_accuracy: 0.2407
Epoch 5/100
68/68 [==============================] - 4s 62ms/step - loss: 2.6084 - categorical_accuracy: 0.2054 - accuracy: 0.2054 - val_loss: 2.4641 - val_categorical_

In [12]:
model.save('model.h5')

/Users/khanhle/Documents/work/iscv/iscv-machine/myconda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
